# Emissions units with Pint

In this notebook we give some examples of how units are handled in SCMData and are built on top of the [Pint](https://github.com/hgrecco/pint) package.

In [1]:
# NBVAL_IGNORE_OUTPUT
import warnings
import traceback

import pint
from pint.errors import DimensionalityError

from scmdata.units import UnitConverter

<IPython.core.display.Javascript object>

pyam - INFO: Running in a notebook, setting `pyam` logging level to `logging.INFO` and adding stderr handler


## UnitConverter

The `UnitConverter` class handles all unit conversions for us. It is used as shown.

In [2]:
uc = UnitConverter("GtC/yr", "Mt CO2 / yr")
uc.convert_from(1)

3666.666666666667

In [3]:
uc.convert_to(1)

0.0002727272727272727

## Pint Unit Registry

The `unit_registry` which sits underneath all conversions can be accessed via `UnitConverter`'s `unit_registry` property. Nevertheless, you should have no need to access it directly.

In [4]:
unit_registry = uc.unit_registry

Having accessed the `unit_registry`, all the units available in SCMData can be shown like so.

In [5]:
# NBVAL_IGNORE_OUTPUT
dir(unit_registry)

['A',
 'A_90',
 'A_US',
 'A_it',
 'At',
 'B',
 'BC',
 'BDFT',
 'BF',
 'BTU',
 'Ba',
 'Bd',
 'Bi',
 'Bq',
 'Btu',
 'Btu_iso',
 'Btu_it',
 'Btu_th',
 'C',
 'C10F18',
 'C2F6',
 'C3F8',
 'C4F10',
 'C5F12',
 'C6F14',
 'C7F16',
 'C8F18',
 'CC3F6',
 'CC4F8',
 'CCL4',
 'CCl4',
 'CF4',
 'CFC11',
 'CFC113',
 'CFC114',
 'CFC115',
 'CFC12',
 'CFC13',
 'CH2CL2',
 'CH2Cl2',
 'CH3BR',
 'CH3Br',
 'CH3CCL3',
 'CH3CCl3',
 'CH3CL',
 'CH3Cl',
 'CH4',
 'CHCL3',
 'CHCl3',
 'CO',
 'CO2',
 'C_90',
 'Ci',
 'Cl',
 'D',
 'DPI',
 'Da',
 'ECC',
 'EC_therm',
 'E_h',
 'Eh',
 'F',
 'FBM',
 'F_90',
 'Fr',
 'G',
 'G_0',
 'Gal',
 'Gb',
 'Gy',
 'H',
 'H2O',
 'HALON1201',
 'HALON1202',
 'HALON1211',
 'HALON1301',
 'HALON2402',
 'HCFC123',
 'HCFC124',
 'HCFC141B',
 'HCFC141b',
 'HCFC142B',
 'HCFC142b',
 'HCFC21',
 'HCFC22',
 'HCFC225CA',
 'HCFC225CB',
 'HCFC225ca',
 'HCFC225cb',
 'HCFE235DA2',
 'HCFE235da2',
 'HFC125',
 'HFC134',
 'HFC134A',
 'HFC134a',
 'HFC143',
 'HFC143A',
 'HFC143a',
 'HFC152',
 'HFC152A',
 'HFC152a',


## Using Pint Directly

For completeness, below we show how to use pint directly. Note that all of these operations are used by `UnitConverter` so the user shouldn't ever have to access pint in this way.

With the `unit_registry`, we can also create Pint variables/arrays which are unit aware.

In [6]:
one_carbon = 1 * unit_registry("C")
one_carbon

1 <Unit('C')>

In [7]:
type(one_carbon)

pint.quantity.build_quantity_class.<locals>.Quantity

In [8]:
one_co2 = 1 * unit_registry.CO2
three_n2o = 3 * unit_registry.N2O

Pint quantities also print in an intuitive way.

In [9]:
print(one_co2)
print(three_n2o)

1 CO2
3 N2O


We can convert them to base units or to each other.

In [10]:
print(one_carbon.to_base_units())
print(one_co2.to("C"))
print(three_n2o.to("N"))

1 C
0.2727272727272727 C
0.9545454545454546 N


Operations are units aware.

In [11]:
print(one_carbon + one_co2)
print(one_carbon * one_co2)
print((one_carbon * one_co2).to_base_units())
print(one_carbon / one_co2)
print((one_carbon / one_co2).to_base_units())

1.2727272727272727 C
1 C * CO2
0.2727272727272727 C ** 2
1.0 C / CO2
3.666666666666667 dimensionless


If we have compound units (e.g. emissions units which are [mass] * [substance] / [time]), we can convert any bit of the unit we want.

In [12]:
eg1 = 1 * unit_registry("Mt") * unit_registry("C") / unit_registry("yr")
print(eg1)
eg2 = 5 * unit_registry("t") * unit_registry("CO2") / unit_registry("s")
print(eg2)

1.0 C * megametric_ton / a
5.0 CO2 * metric_ton / second


In [13]:
print(eg1.to("Gt CO2 / day"))
print(eg2.to("Gt C / yr"))

1.0038786219484371e-05 CO2 * gigametric_ton / day
0.04303309090909091 C * gigametric_ton / a


## Contexts

With a context, we can use metric conversion definitions to do emissions conversions that would otherwise raise a `DimensionalityError`. For example, converting CO2 to N2O using AR4GWP100 (where 298 tCO2 = 1 tN2O).

In [14]:
ar4gwp100uc = UnitConverter("N2O", "CO2", context="AR4GWP100")
ar4gwp100uc.convert_from(1)

298.0

In [15]:
ar4gwp100uc = UnitConverter("N2O", "CH4", context="AR4GWP100")
ar4gwp100uc.convert_from(1)

11.920000000000002

We can see which contexts we have (which we can use for e.g. metric conversions).

In [16]:
ar4gwp100uc.contexts

['Gaussian',
 'Gau',
 'ESU',
 'esu',
 'spectroscopy',
 'sp',
 'boltzmann',
 'energy',
 'chemistry',
 'chem',
 'textile',
 'CH4_conversions',
 'NOx_conversions',
 'SARGWP100',
 'AR4GWP100',
 'AR5GWP100']

Such context dependent conversions can also be done directly with Pint.

In [17]:
base = 1 * unit_registry("N2O")
with unit_registry.context("AR4GWP100"):
    print(one_carbon)
    print(one_carbon.to("CO2"))
    print(
        one_carbon.to("N") + three_n2o
    )  # I am not sure why you need to force the conversion of `a` first...

1 C
3.666666666666667 CO2
0.9584604433597722 N


Without a context to tell us about metrics, if we try to do an invalid conversion, a `DimensionalityError` will be raised.

In [18]:
try:
    ar4gwp100uc = UnitConverter("N2O", "CO2")
    ar4gwp100uc.convert_from(1)
except DimensionalityError:
    traceback.print_exc(limit=0, chain=False)

Traceback (most recent call last):
pint.errors.DimensionalityError: Cannot convert from 'N2O' ([nitrogen]) to 'CO2' ([carbon])


In [19]:
try:
    base.to("CO2")
except DimensionalityError:
    traceback.print_exc(limit=0, chain=False)

Traceback (most recent call last):
pint.errors.DimensionalityError: Cannot convert from 'N2O' ([nitrogen]) to 'CO2' ([carbon])


If the context you use does not have the conversion you request, a warning will be raised. Any subsequent conversions will result in NaN's.

In [20]:
# modify the way the warning appears to remove the path,
# thank you https://stackoverflow.com/a/26433913
def custom_formatting(
    message, category, filename, lineno, file=None, line=None
):
    return "{}: {}\n".format(category.__name__, message)


warnings.formatwarning = custom_formatting

ucnan = UnitConverter("N2O", "Halon2402", context="SARGWP100")
ucnan.convert_from(1)

nan